In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-08-14 23:53:34.494880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 23:53:34.703803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-14 23:53:34.703827: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-14 23:53:35.467349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
data_folder = "../../data/BARseq/BARseq_Perturbed/"
results_folder = "./results/Spateo_global/"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
slices = [ad.read_h5ad(data_folder + "slice_{:0>2d}.h5ad".format(i)) for i in range(1, 41)]

In [4]:
transformation = np.load(f'./results/stat_results/Spateo_transformation.npy', allow_pickle=True)
align_Rotation = transformation.item()['align_Rotation']
align_translation = transformation.item()['align_translation']

In [5]:
slices[0]

AnnData object with n_obs × n_vars = 34029 × 114
    obs: 'cluster', 'slice_id', 'simplify_cluster'
    obsm: 'align_spatial', 'perturbed_spatial', 'spatial', 'spatial_CCF', 'spatial_raw'

In [6]:
from tqdm import tqdm
cur_R = np.eye(2)
cur_t = np.zeros(2)
for i in tqdm(range(len(slices))):
    slices[i].obsm['align_spatial_2D'] = slices[i].obsm['perturbed_spatial'] @ cur_R.T + cur_t
    if i < len(slices)-1:
        R = align_Rotation[i]
        t = align_translation[i]
        cur_t = t @ cur_R.T + cur_t
        cur_R = cur_R @ R

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 608.04it/s]


In [8]:
slices_ref = []
sampling_num = 20000
for s in slices:
    sampline_idx = np.random.choice(s.shape[0], sampling_num, replace=False) if s.shape[0] > sampling_num else np.arange(s.shape[0])
    slices_ref.append(s[sampline_idx,:].copy())

In [9]:
key_added = "global_align_spatial"
global_align_slices = st.align.morpho_global_optimization(
    models=slices_ref,
    spatial_key='align_spatial_2D',
    key_added="global_align_spatial",
    device="0",
    mode="SN-S",
    max_iter_global=5,
    max_iter=200,
    verbose=False,
    partial_robust_level=10,
    beta=0.05,
    lambdaVF=100,
    vis_optimization=False,
    neighbor_size=2,
)

  0%|                                                                                                                                | 0/40 [00:00<?, ?it/s]/home/ylu/project/spateo-release_new/spateo_dev/alignment/methods/utils.py:1400: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

  5%|██████                                                                                                                  | 2/40 [00:02<00:52,  1.37s/it]/home/ylu/project/spateo-release_new/spateo_dev/alignment/methods/utils.py:1400: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

  8%|█████████                                                                                                               | 3/4

Global optimization successfully!


In [ ]:
# save the transformation
align_Rotation = []
align_translation = []
for s in global_align_slices:
    R, t=solve_RT_by_correspondence(s.obsm['global_align_spatial'], s.obsm['spatial_2D'])
    align_Rotation.append(R)
    align_translation.append(t)
np.save('./results/stats_results/Spateo_global_transformation.npy', {'align_Rotation': align_Rotation, 'align_translation': align_translation}, allow_pickle=True)